In [ ]:
import csv
import matplotlib.pyplot as plt
import nltk
import pandas
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
# import nltk
# nltk.download('stopwords')

pandas.set_option('display.max_rows', 1000)

# loading the dataset
dataset = pandas.read_csv("labeled01.csv",sep=',',usecols=['tweet','label'])

print(type(dataset))
print(f"Number of tweets: {len(dataset)}")
display(dataset)

In [ ]:
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [ ]:
# SIMPLE PIPELINE WITHOUT ANY TEST ON ngram_range AND CONSIDERING ONLY ACCURACY AS METRIC
# SE GLI ALTRI TEST VANNO BENE QUESTO SI PUO' ANCHE ELIMINARE

text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
folds =  10
#calculating accuracies in cross-validation
scores = cross_val_score(text_clf, dataset.tweet, dataset.label, cv=folds)
("Accuracy MultinomialNB : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#print(scores)

In [158]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average = 'weighted', zero_division = 0),
           'recall' : make_scorer(recall_score, average = 'macro'), 
           'f1_score' : make_scorer(f1_score, average = 'macro')
          }

def print_metrics(scores):
    print(f'accuracy on test set: {scores["test_accuracy"].mean():.3} +/- {scores["test_accuracy"].std()*2:.3}')
    print(f'precision on test set: {scores["test_precision"].mean():.3} +/- {scores["test_precision"].std()*2:.3}')
    print(f'recall on test set: {scores["test_recall"].mean():.3} +/- {scores["test_recall"].std()*2:.3}')
    print(f'f1-score on test set: {scores["test_f1_score"].mean():.3} +/- {scores["test_f1_score"].std()*2:.3}')

In [159]:
max_ngram = 2

print('NAIVE BAYES CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', MultinomialNB()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)

NAIVE BAYES CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.505 +/- 0.176
precision on test set: 0.517 +/- 0.172
recall on test set: 0.493 +/- 0.181
f1-score on test set: 0.487 +/- 0.192
------- ngram = (1, 2) -------
accuracy on test set: 0.503 +/- 0.186
precision on test set: 0.518 +/- 0.192
recall on test set: 0.492 +/- 0.19
f1-score on test set: 0.489 +/- 0.202
------- ngram = (2, 2) -------
accuracy on test set: 0.445 +/- 0.116
precision on test set: 0.443 +/- 0.143
recall on test set: 0.428 +/- 0.113
f1-score on test set: 0.411 +/- 0.125


In [160]:
print('DECISION TREE CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', tree.DecisionTreeClassifier()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)

DECISION TREE CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.428 +/- 0.126
precision on test set: 0.425 +/- 0.135
recall on test set: 0.423 +/- 0.132
f1-score on test set: 0.42 +/- 0.137
------- ngram = (1, 2) -------
accuracy on test set: 0.437 +/- 0.155
precision on test set: 0.437 +/- 0.152
recall on test set: 0.435 +/- 0.159
f1-score on test set: 0.433 +/- 0.156
------- ngram = (2, 2) -------
accuracy on test set: 0.392 +/- 0.15
precision on test set: 0.395 +/- 0.172
recall on test set: 0.384 +/- 0.147
f1-score on test set: 0.374 +/- 0.154


In [161]:
print('SVM CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', svm.LinearSVC()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring=scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)
            #print(scores)

SVM CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.493 +/- 0.156
precision on test set: 0.491 +/- 0.159
recall on test set: 0.491 +/- 0.16
f1-score on test set: 0.488 +/- 0.159
------- ngram = (1, 2) -------
accuracy on test set: 0.517 +/- 0.181
precision on test set: 0.516 +/- 0.181
recall on test set: 0.514 +/- 0.18
f1-score on test set: 0.511 +/- 0.18
------- ngram = (2, 2) -------
accuracy on test set: 0.398 +/- 0.139
precision on test set: 0.401 +/- 0.167
recall on test set: 0.389 +/- 0.133
f1-score on test set: 0.383 +/- 0.132


In [ ]:
"""
from sklearn.metrics import precision_recall_fscore_support
def overall_average_score(actual,prediction):
    precision, recall, f1_score, support = precision_recall_fscore_support(
        actual, prediction, average=None)
    print(type(f1_score))
    return f1_score.mean()

custom_scorer = make_scorer(overall_average_score)


text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = custom_scorer)
print(scores) """